In [24]:

import pandas as pd 
import sqlite3


In [6]:
base_de_datos = "travel.sqlite"

### TABLA 1

## aircrafts_data (datos_de_aeronaves)

In [3]:
#Conexion a base de datos
conn = sqlite3.connect(base_de_datos)

# Consulta para obtener solo la tabla 'aircrafts_data'
datos_de_aeronaves = pd.read_sql("""
                            SELECT *
                            FROM aircrafts_data;
                            """, conn)

# Mostrar el contenido de la tabla 'aircrafts_data'
print(datos_de_aeronaves)

  aircraft_code                                              model  range
0           773    {"en": "Boeing 777-300", "ru": "Боинг 777-300"}  11100
1           763    {"en": "Boeing 767-300", "ru": "Боинг 767-300"}   7900
2           SU9  {"en": "Sukhoi Superjet-100", "ru": "Сухой Суп...   3000
3           320  {"en": "Airbus A320-200", "ru": "Аэробус A320-...   5700
4           321  {"en": "Airbus A321-200", "ru": "Аэробус A321-...   5600
5           319  {"en": "Airbus A319-100", "ru": "Аэробус A319-...   6700
6           733    {"en": "Boeing 737-300", "ru": "Боинг 737-300"}   4200
7           CN1  {"en": "Cessna 208 Caravan", "ru": "Сессна 208...   1200
8           CR2  {"en": "Bombardier CRJ-200", "ru": "Бомбардье ...   2700


In [54]:
import pandas as pd

# Datos corregidos de la tabla
data = {
    'index': [0, 1, 2, 3, 4, 5, 6, 7, 8],
    'aircraft_code': ['773', '763', 'SU9', '320', '321', '319', '733', 'CN1', 'CR2'],
    'model': [
        {"en": "Boeing 777-300"},
        {"en": "Boeing 767-300"},
        {"en": "Sukhoi Superjet-100"},
        {"en": "Airbus A320-200"},
        {"en": "Airbus A321-200"},
        {"en": "Airbus A319-100"},
        {"en": "Boeing 737-300"},
        {"en": "Cessna 208 Caravan"},
        {"en": "Bombardier CRJ-200"}
    ],
    'range': [11100, 7900, 3000, 5700, 5600, 6700, 4200, 1200, 2700]
}

# Convertir a DataFrame
df = pd.DataFrame(data)

# Expandir la columna 'model' en una nueva columna 'model_en'
df['model_en'] = df['model'].apply(lambda x: x['en'])

# Eliminar la columna original 'model'
df.drop(columns='model', inplace=True)

# Guardar el DataFrame en un archivo CSV
df.to_csv('aircraft_data.csv', index=False)

# Mostrar el DataFrame final
print(df)





   index aircraft_code  range             model_en
0      0           773  11100       Boeing 777-300
1      1           763   7900       Boeing 767-300
2      2           SU9   3000  Sukhoi Superjet-100
3      3           320   5700      Airbus A320-200
4      4           321   5600      Airbus A321-200
5      5           319   6700      Airbus A319-100
6      6           733   4200       Boeing 737-300
7      7           CN1   1200   Cessna 208 Caravan
8      8           CR2   2700   Bombardier CRJ-200


### TABLA 2

### airports_data (datos_de_aeropuertos)

In [59]:
# Conexión a la base de datos (ajustar la ruta de 'base_de_datos')
conn = sqlite3.connect(base_de_datos)

# Consulta para obtener la tabla 'airports_data'
datos_de_aeropuertos = pd.read_sql("""
                            SELECT *
                            FROM airports_data;
                            """, conn)

# Cerrar la conexión a la base de datos
conn.close()

# Función para extraer el valor de la clave 'en' de un diccionario
def extraer_en(valor):
    if isinstance(valor, dict):
        return valor.get('en', None)
    return valor

# Función para separar las coordenadas (latitud y longitud)
def separar_coordenadas(coordenadas):
    if isinstance(coordenadas, str):
        coords = coordenadas.split(',')
        if len(coords) == 2:
            return pd.Series({'longitude': coords[0], 'latitude': coords[1]})
    return pd.Series({'longitude': None, 'latitude': None})

# Aplicar la extracción de 'en' en las columnas que contienen diccionarios
datos_de_aeropuertos['airport_name'] = datos_de_aeropuertos['airport_name'].apply(extraer_en)
datos_de_aeropuertos['city'] = datos_de_aeropuertos['city'].apply(extraer_en)

# Aplicar la separación de coordenadas
coordenadas_separadas = datos_de_aeropuertos['coordinates'].apply(separar_coordenadas)

# Añadir las nuevas columnas al DataFrame original
datos_de_aeropuertos = pd.concat([datos_de_aeropuertos, coordenadas_separadas], axis=1)

# Eliminar la columna original de coordenadas si ya no es necesaria
datos_de_aeropuertos.drop(columns=['coordinates'], inplace=True)

# Mostrar el DataFrame final
print("\nNuevo DataFrame procesado:")
print(datos_de_aeropuertos)

# Opcional: Guardar el resultado en un nuevo archivo CSV
datos_de_aeropuertos.to_csv('airports_data_proces.csv', index=False)





Nuevo DataFrame procesado:
    airport_code                                       airport_name  \
0            YKS          {"en": "Yakutsk Airport", "ru": "Якутск"}   
1            MJZ            {"en": "Mirny Airport", "ru": "Мирный"}   
2            KHV  {"en": "Khabarovsk-Novy Airport", "ru": "Хабар...   
3            PKC        {"en": "Yelizovo Airport", "ru": "Елизово"}   
4            UUS  {"en": "Yuzhno-Sakhalinsk Airport", "ru": "Хом...   
..           ...                                                ...   
99           MMK       {"en": "Murmansk Airport", "ru": "Мурманск"}   
100          ABA           {"en": "Abakan Airport", "ru": "Абакан"}   
101          BAX         {"en": "Barnaul Airport", "ru": "Барнаул"}   
102          AAQ  {"en": "Anapa Vityazevo Airport", "ru": "Витяз...   
103          CNN         {"en": "Chulman Airport", "ru": "Чульман"}   

                                                  city          timezone  \
0                    {"en": "Yakutsk", "ru"

In [73]:
# Cargar el archivo CSV
datos_de_aeropuertos = pd.read_csv('airports_data_proces.csv')

# Función para extraer el valor de la clave 'en' de un diccionario
def extraer_en(valor):
    if isinstance(valor, str) and valor.startswith("{"):
        try:
            # Convertir la cadena de texto en un diccionario y obtener el valor 'en'
            valor_dict = eval(valor)
            return valor_dict.get('en', None)
        except:
            return valor
    return valor



# Aplicar la extracción de 'en' en las columnas que contienen diccionarios
datos_de_aeropuertos['airport_name'] = datos_de_aeropuertos['airport_name'].apply(extraer_en)
datos_de_aeropuertos['city'] = datos_de_aeropuertos['city'].apply(extraer_en)



# Guardar el DataFrame resultante en un nuevo archivo CSV
datos_de_aeropuertos.to_csv('airports_data_normalizado.csv', index=False)

print("Datos normalizados y guardados en 'airports_data_normalizado.csv'")


Datos normalizados y guardados en 'airports_data_normalizado.csv'


### TABLA 3

### boarding_passes (tarjetas_de_embarque)

In [70]:
conn = sqlite3.connect(base_de_datos)

# Consulta para obtener solo la tabla 'boarding_passes'
tarjetas_de_embarque = pd.read_sql("""
                            SELECT *
                            FROM boarding_passes;
                            """, conn)

# Cerrar la conexión a la base de datos
conn.close()

# Mostrar el contenido de la tabla 'boarding_passes'
print("Contenido de la tabla 'boarding_passes':")
print(tarjetas_de_embarque)

#recorrer los datos y crear un nuevo DataFrame a partir de las columnas de 'datos_de_aeropuertos'
# Crear un DataFrame vacío
df = pd.DataFrame()

# Recorrer las columnas del DataFrame 'datos_de_aeropuertos' y añadirlas a 'df'
for column in tarjetas_de_embarque.columns:
    df[column] = tarjetas_de_embarque[column]

# Mostrar el nuevo DataFrame
print("\nNuevo DataFrame copiado:")
print(df)

Contenido de la tabla 'boarding_passes':
            ticket_no  flight_id  boarding_no seat_no
0       0005435212351      30625            1      2D
1       0005435212386      30625            2      3G
2       0005435212381      30625            3      4H
3       0005432211370      30625            4      5D
4       0005435212357      30625            5     11A
...               ...        ...          ...     ...
579681  0005434302871      19945           85     20F
579682  0005432892791      19945           86     21C
579683  0005434302869      19945           87     20E
579684  0005432802476      19945           88     21F
579685  0005432802482      19945           89     21E

[579686 rows x 4 columns]

Nuevo DataFrame copiado:
            ticket_no  flight_id  boarding_no seat_no
0       0005435212351      30625            1      2D
1       0005435212386      30625            2      3G
2       0005435212381      30625            3      4H
3       0005432211370      30625          

In [71]:
# Verificar el número de valores nulos en cada columna
valores_nulos = tarjetas_de_embarque.isnull().sum()
print("Valores nulos por columna:")
print(valores_nulos)


Valores nulos por columna:
ticket_no      0
flight_id      0
boarding_no    0
seat_no        0
dtype: int64


In [67]:
# Verificar el número de valores únicos en cada columna
valores_unicos = df.nunique()
print("\nValores únicos por columna:")
print(valores_unicos)



Valores únicos por columna:
ticket_no      238834
flight_id       11518
boarding_no       374
seat_no           461
dtype: int64


In [72]:
# Guardar el DataFrame 'df' en un archivo CSV
df.to_csv('tarjetas_de_embarque.csv', index=False)

print("El DataFrame se ha guardado correctamente en 'tarjetas_de_embarque.csv'.")


El DataFrame se ha guardado correctamente en 'tarjetas_de_embarque.csv'.


### TABLA 4

### bookings (reservas)

In [75]:

conn = sqlite3.connect(base_de_datos)

# Consulta para obtener solo la tabla 'bookings'
reservas = pd.read_sql("""
                            SELECT *
                            FROM bookings;
                            """, conn)

# Cerrar la conexión a la base de datos
conn.close()

# Mostrar el contenido de la tabla 'bookings'
print("Contenido de la tabla 'bookings':")
print(reservas)

# Si quieres recorrer los datos y crear un nuevo DataFrame a partir de las columnas de 'datos_de_aeropuertos'
# Crear un DataFrame vacío
df_reservas = pd.DataFrame()

# Recorrer las columnas del DataFrame 'datos_de_aeropuertos' y añadirlas a 'df'
for column in reservas.columns:
    df[column] = reservas[column]

# Mostrar el nuevo DataFrame
print("\nNuevo DataFrame copiado:")
print(df)

Contenido de la tabla 'bookings':
       book_ref               book_date  total_amount
0        00000F  2017-07-05 03:12:00+03        265700
1        000012  2017-07-14 09:02:00+03         37900
2        000068  2017-08-15 14:27:00+03         18100
3        000181  2017-08-10 13:28:00+03        131800
4        0002D8  2017-08-07 21:40:00+03         23600
...         ...                     ...           ...
262783   FFFEF3  2017-07-17 07:23:00+03         56000
262784   FFFF2C  2017-08-08 05:55:00+03         10800
262785   FFFF43  2017-07-20 20:42:00+03         78500
262786   FFFFA8  2017-08-08 04:45:00+03         28800
262787   FFFFF7  2017-07-01 22:12:00+03         73600

[262788 rows x 3 columns]

Nuevo DataFrame copiado:
       book_ref               book_date  total_amount
0        00000F  2017-07-05 03:12:00+03        265700
1        000012  2017-07-14 09:02:00+03         37900
2        000068  2017-08-15 14:27:00+03         18100
3        000181  2017-08-10 13:28:00+03        13

In [77]:
# Verificar el número de valores nulos en cada columna
valores_nulos_reservas = reservas.isnull().sum()
print("Valores nulos por columna:")
print(valores_nulos_reservas)

Valores nulos por columna:
book_ref        0
book_date       0
total_amount    0
dtype: int64


In [78]:
# Verificar el número de valores únicos en cada columna
valores_unicos_reservas = df.nunique()
print("\nValores únicos por columna:")
print(valores_unicos_reservas)


Valores únicos por columna:
book_ref        262788
book_date        68631
total_amount      3926
dtype: int64


In [79]:
# Guardar el DataFrame 'df' en un archivo CSV
df.to_csv('reservas.csv', index=False)

print("El DataFrame se ha guardado correctamente en 'reservas.csv'.")

El DataFrame se ha guardado correctamente en 'reservas.csv'.


### TABLA 5

### flights (vuelos)

In [80]:

conn = sqlite3.connect(base_de_datos)

# Consulta para obtener solo la tabla 'flights'
vuelos = pd.read_sql("""
                            SELECT *
                            FROM flights;
                            """, conn)

# Cerrar la conexión a la base de datos
conn.close()

# Mostrar el contenido de la tabla 'flights'
print("Contenido de la tabla 'flights':")
print(vuelos)

# Si quieres recorrer los datos y crear un nuevo DataFrame a partir de las columnas de 'datos_de_aeropuertos'
# Crear un DataFrame vacío
df = pd.DataFrame()

# Recorrer las columnas del DataFrame 'datos_de_aeropuertos' y añadirlas a 'df'
for column in vuelos.columns:
    df[column] = vuelos[column]

# Mostrar el nuevo DataFrame
print("\nNuevo DataFrame copiado:")
print(df)

Contenido de la tabla 'flights':
       flight_id flight_no     scheduled_departure       scheduled_arrival  \
0           1185    PG0134  2017-09-10 09:50:00+03  2017-09-10 14:55:00+03   
1           3979    PG0052  2017-08-25 14:50:00+03  2017-08-25 17:35:00+03   
2           4739    PG0561  2017-09-05 12:30:00+03  2017-09-05 14:15:00+03   
3           5502    PG0529  2017-09-12 09:50:00+03  2017-09-12 11:20:00+03   
4           6938    PG0461  2017-09-04 12:25:00+03  2017-09-04 13:20:00+03   
...          ...       ...                     ...                     ...   
33116      33117    PG0063  2017-08-02 19:25:00+03  2017-08-02 20:10:00+03   
33117      33118    PG0063  2017-07-28 19:25:00+03  2017-07-28 20:10:00+03   
33118      33119    PG0063  2017-09-08 19:25:00+03  2017-09-08 20:10:00+03   
33119      33120    PG0063  2017-08-01 19:25:00+03  2017-08-01 20:10:00+03   
33120      33121    PG0063  2017-08-26 19:25:00+03  2017-08-26 20:10:00+03   

      departure_airport arriva

In [82]:
# Verificar el número de valores nulos en cada columna
valores_nulos_vuelos = vuelos.isnull().sum()
print("Valores nulos por columna:")
print(valores_nulos_vuelos)

Valores nulos por columna:
flight_id              0
flight_no              0
scheduled_departure    0
scheduled_arrival      0
departure_airport      0
arrival_airport        0
status                 0
aircraft_code          0
actual_departure       0
actual_arrival         0
dtype: int64


In [83]:
# Verificar el número de valores únicos en cada columna
valores_unicos_vuelos = df.nunique()
print("\nValores únicos por columna:")
print(valores_unicos_vuelos)


Valores únicos por columna:
flight_id              33121
flight_no                710
scheduled_departure    10365
scheduled_arrival       9648
departure_airport        104
arrival_airport          104
status                     6
aircraft_code              8
actual_departure       12108
actual_arrival         11851
dtype: int64


In [84]:
# Guardar el DataFrame 'df' en un archivo CSV
df.to_csv('vuelos.csv', index=False)

print("El DataFrame se ha guardado correctamente en 'vuelos.csv'.")

El DataFrame se ha guardado correctamente en 'vuelos.csv'.


### TABLA 6

### Seats (Asientos)

In [85]:

conn = sqlite3.connect(base_de_datos)

# Consulta para obtener solo la tabla 'seats'
asientos = pd.read_sql("""
                            SELECT *
                            FROM seats;
                            """, conn)

# Cerrar la conexión a la base de datos
conn.close()

# Mostrar el contenido de la tabla 'seats'
print("Contenido de la tabla 'seats':")
print(datos_de_aeropuertos)

# Si quieres recorrer los datos y crear un nuevo DataFrame a partir de las columnas de 'datos_de_aeropuertos'
# Crear un DataFrame vacío
df = pd.DataFrame()

# Recorrer las columnas del DataFrame 'datos_de_aeropuertos' y añadirlas a 'df'
for column in asientos.columns:
    df[column] = asientos[column]

# Mostrar el nuevo DataFrame
print("\nNuevo DataFrame copiado:")
print(df)

Contenido de la tabla 'seats':
       book_ref               book_date  total_amount
0        00000F  2017-07-05 03:12:00+03        265700
1        000012  2017-07-14 09:02:00+03         37900
2        000068  2017-08-15 14:27:00+03         18100
3        000181  2017-08-10 13:28:00+03        131800
4        0002D8  2017-08-07 21:40:00+03         23600
...         ...                     ...           ...
262783   FFFEF3  2017-07-17 07:23:00+03         56000
262784   FFFF2C  2017-08-08 05:55:00+03         10800
262785   FFFF43  2017-07-20 20:42:00+03         78500
262786   FFFFA8  2017-08-08 04:45:00+03         28800
262787   FFFFF7  2017-07-01 22:12:00+03         73600

[262788 rows x 3 columns]

Nuevo DataFrame copiado:
     aircraft_code seat_no fare_conditions
0              319      2A        Business
1              319      2C        Business
2              319      2D        Business
3              319      2F        Business
4              319      3A        Business
...       

In [86]:
# Verificar el número de valores nulos en cada columna
valores_nulos_asientos = asientos.isnull().sum()
print("Valores nulos por columna:")
print(valores_nulos_asientos)

Valores nulos por columna:
aircraft_code      0
seat_no            0
fare_conditions    0
dtype: int64


In [87]:
# Verificar el número de valores únicos en cada columna
valores_unicos_asientos = asientos.nunique()
print("\nValores únicos por columna:")
print(valores_unicos_asientos)


Valores únicos por columna:
aircraft_code        9
seat_no            461
fare_conditions      3
dtype: int64


In [88]:
# Guardar el DataFrame 'df' en un archivo CSV
df.to_csv('asientos.csv', index=False)

print("El DataFrame se ha guardado correctamente en 'asientos.csv'.")

El DataFrame se ha guardado correctamente en 'asientos.csv'.


### TABLA 7

### ticket_flights (boletos_de_Avion)

In [98]:

conn = sqlite3.connect(base_de_datos)

# Consulta para obtener solo la tabla 'ticket_flights'
boletos_de_avion = pd.read_sql("""
                            SELECT *
                            FROM ticket_flights;
                            """, conn)

# Cerrar la conexión a la base de datos
conn.close()

# Mostrar el contenido de la tabla 'seats'
print("Contenido de la tabla 'ticket_flights':")
print(datos_de_aeropuertos)

# Si quieres recorrer los datos y crear un nuevo DataFrame a partir de las columnas de 'datos_de_aeropuertos'
# Crear un DataFrame vacío
df = pd.DataFrame()

# Recorrer las columnas del DataFrame 'datos_de_aeropuertos' y añadirlas a 'df'
for column in boletos_de_avion.columns:
    df[column] = boletos_de_avion[column]

# Mostrar el nuevo DataFrame
print("\nNuevo DataFrame copiado:")
print(df)


Contenido de la tabla 'ticket_flights':
       book_ref               book_date  total_amount
0        00000F  2017-07-05 03:12:00+03        265700
1        000012  2017-07-14 09:02:00+03         37900
2        000068  2017-08-15 14:27:00+03         18100
3        000181  2017-08-10 13:28:00+03        131800
4        0002D8  2017-08-07 21:40:00+03         23600
...         ...                     ...           ...
262783   FFFEF3  2017-07-17 07:23:00+03         56000
262784   FFFF2C  2017-08-08 05:55:00+03         10800
262785   FFFF43  2017-07-20 20:42:00+03         78500
262786   FFFFA8  2017-08-08 04:45:00+03         28800
262787   FFFFF7  2017-07-01 22:12:00+03         73600

[262788 rows x 3 columns]

Nuevo DataFrame copiado:
             ticket_no  flight_id fare_conditions  amount
0        0005432159776      30625        Business   42100
1        0005435212351      30625        Business   42100
2        0005435212386      30625        Business   42100
3        0005435212381     

In [103]:
# Resumen general de valores nulos y únicos por columna
resumen = pd.DataFrame({
    'Valores Nulos': df.isnull().sum(),
    'Valores Únicos': df.nunique(),
    'Total de Registros': len(df)
})

print("\nResumen general:")
print(resumen)



Resumen general:
                 Valores Nulos  Valores Únicos  Total de Registros
ticket_no                    0          366733             1045726
flight_id                    0           22226             1045726
fare_conditions              0               3             1045726
amount                       0             338             1045726


In [104]:
# Guardar el DataFrame 'df' en un archivo CSV
df.to_csv('boletos_de_avion.csv', index=False)

print("El DataFrame se ha guardado correctamente en 'boletos_de_avion.csv'.")

El DataFrame se ha guardado correctamente en 'boletos_de_avion.csv'.


### TABLA 8

### Tickets (Boletos)

In [114]:
# Conectar a la base de datos
conn = sqlite3.connect(base_de_datos)

# Consulta para obtener la tabla 'tickets'
boletos = pd.read_sql("""
                            SELECT *
                            FROM tickets;
                            """, conn)

# Cerrar la conexión a la base de datos
conn.close()

# Mostrar el contenido de la tabla 'tickets'
print("Contenido de la tabla 'tickets':")
print(boletos.head())  # Mostrar las primeras filas para verificar los datos

# Aquí el DataFrame 'boletos' ya está completo, no es necesario crear otro DataFrame vacío

# Guardar el DataFrame 'boletos' en un archivo CSV (si es necesario)
boletos.to_csv('boletos_tickets.csv', index=False)

print("\nEl DataFrame se ha guardado correctamente en 'boletos_tickets.csv'.")


Contenido de la tabla 'tickets':
       ticket_no book_ref passenger_id
0  0005432000987   06B046  8149 604011
1  0005432000988   06B046  8499 420203
2  0005432000989   E170C3  1011 752484
3  0005432000990   E170C3  4849 400049
4  0005432000991   F313DD  6615 976589

El DataFrame se ha guardado correctamente en 'boletos_tickets.csv'.


In [115]:
# Resumen general de valores nulos y únicos por columna
conteo = pd.DataFrame({
    'Valores Nulos': boletos.isnull().sum(),
    'Valores Únicos': boletos.nunique(),
    'Total de Registros': len(boletos)
})

print("\nResumen general:")
print(conteo)


Resumen general:
              Valores Nulos  Valores Únicos  Total de Registros
ticket_no                 0          366733              366733
book_ref                  0          262788              366733
passenger_id              0          366733              366733
